---
layout: post
title: N-Body
---

In [1]:
from pathlib import Path
import os
import functools

from IPython.display import HTML, Image
import matplotlib.pyplot as plt
from numpy import *
from celluloid import Camera
import matplotlib.patches as patches
from scipy.integrate import odeint, solve_ivp
from simple_pid import PID

ROOT = Path("./assets/img/")

if not os.path.exists(ROOT):
    os.makedirs(ROOT)

N-Body problem.

$$
\frac{d^2 r}{dt^2} = -G \sum_{i=1}^N \frac{m_i}{|r_i|^2} r_i
$$

In [2]:
G = 6.2#*10**-11
MASS = array([10, 20, 30])

In [5]:
def motion_step(s0, t):

    s0 = s0.reshape(-1, 6)
    N = s0.shape[0]

    s = []
    for i in range(N):
        
        ri0, vi0 = s0[i,:3], s0[i,3:]        
        dr = vi0
        dv = 0.
        for j in range(N):
            if i != j:
                
                rj0 = s0[j,:3]
                rj = ri0 - rj0
                rj = rj / (linalg.norm(rj)+1e-7)

                dv += -G * MASS[j] * rj / linalg.norm(rj)**2
        s.append(concat([dr, dv], axis=0))

    s = array(s).reshape(-1)
    return s

In [ ]:
def sim_n_body():

    rng = random.default_rng(seed=0)

    T = 5
    t = linspace(0, T, 500)

    s0 = rng.uniform(size=(3, 6))
    s0[:, 3:] = 0.
    
    sol = odeint(motion_step, s0.reshape(-1), t)

    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    camera = Camera(fig)
    
    for step in sol:
        
        step = step.reshape(-1, 6)

        r = step[:,:3]
        ax.scatter(r[:,0], r[:,1], r[:,2], color="black")
        camera.snap()

    anim = camera.animate()
    plt.close()

    gif_path = ROOT / "nbody.gif"  
    anim.save(gif_path, writer="pillow", fps=10)
    return Image(url=gif_path)

sim_n_body()